In [1]:
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
%matplotlib inline

init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

# Importing the dataset
dataset = pd.read_csv('train.csv', delimiter = ',',usecols=["text", "offense_rating"])

print(dataset)

                                                   text  offense_rating
0     TENNESSEE: We're the best state. Nobody even c...            0.20
1     A man inserted an advertisement in the classif...            1.10
2     How many men does it take to open a can of bee...            2.40
3     Told my mom I hit 1200 Twitter followers. She ...            0.00
4     Roses are dead. Love is fake. Weddings are bas...            0.10
...                                                 ...             ...
7995  Lack of awareness of the pervasiveness of raci...            0.25
7996    Why are aspirins white? Because they work sorry            3.85
7997  Today, we Americans celebrate our independence...            0.00
7998  How to keep the flies off the bride at an Ital...            3.00
7999  "Each ounce of sunflower seeds gives you 37% o...            0.00

[8000 rows x 2 columns]


In [2]:
print(len(dataset[dataset['offense_rating'] == '']))

0


There are no cells without an offense_rating value

In [3]:
dataset['offense_rating'].iplot(
    kind='hist',
    bins=50,
    xTitle='Offense Rating',
    linecolor='black',
    yTitle='count',
    title='Offense Rating Distribution')

### The offense rating mean:

In [4]:
mean = dataset['offense_rating'].mean()
print("Mean value for offense rating:", mean)

Mean value for offense rating: 0.5853250000000031


### The offense rating median:

In [5]:
median = dataset['offense_rating'].median()
print("Median value for offense rating:", median)

Median value for offense rating: 0.1


### The offense rating mode:

In [6]:
modeOffense = dataset['offense_rating'].mode()
print("Mode value for offense rating:", modeOffense)

Mode value for offense rating: 0    0.0
dtype: float64


### The offense rating variance:

In [7]:
variance = dataset['offense_rating'].var()
print("Variance value for offense rating:", variance)

Variance value for offense rating: 0.960312183397919


### The offense rating standard deviation:

In [8]:
std = dataset['offense_rating'].std()
print("Standard Deviation value for offense rating:", std)

Standard Deviation value for offense rating: 0.9799551945869357


### The offense rating skewness:

In [9]:
skewness = dataset['offense_rating'].skew()
print("Skewness value for offense rating:", skewness)

Skewness value for offense rating: 2.025170682712623


### Using the Bag of Words:

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dataset.text)
X_train_counts.shape

(8000, 14502)

### Applying TF-IDF:

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(8000, 14502)